<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Util-Functions" data-toc-modified-id="Util-Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Util Functions</a></span><ul class="toc-item"><li><span><a href="#Save-to-pickle" data-toc-modified-id="Save-to-pickle-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Save to pickle</a></span></li><li><span><a href="#Load-Pickle" data-toc-modified-id="Load-Pickle-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load Pickle</a></span></li><li><span><a href="#Open-Zipfile" data-toc-modified-id="Open-Zipfile-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Open Zipfile</a></span></li></ul></li><li><span><a href="#Load-drug_list" data-toc-modified-id="Load-drug_list-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load drug_list</a></span><ul class="toc-item"><li><span><a href="#Load-Root" data-toc-modified-id="Load-Root-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load Root</a></span></li><li><span><a href="#Save-drugs-list" data-toc-modified-id="Save-drugs-list-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Save drugs list</a></span></li><li><span><a href="#Load-preprocessed-file" data-toc-modified-id="Load-preprocessed-file-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Load preprocessed file</a></span></li></ul></li><li><span><a href="#Scrap" data-toc-modified-id="Scrap-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Scrap</a></span><ul class="toc-item"><li><span><a href="#Scrap-drugs-indications" data-toc-modified-id="Scrap-drugs-indications-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Scrap drugs-indications</a></span></li><li><span><a href="#Scap-all-drugs-pages" data-toc-modified-id="Scap-all-drugs-pages-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Scap all drugs pages</a></span></li><li><span><a href="#Scrap-all-indication-pages" data-toc-modified-id="Scrap-all-indication-pages-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Scrap all indication pages</a></span></li><li><span><a href="#save-drugs-and-indications" data-toc-modified-id="save-drugs-and-indications-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>save drugs and indications</a></span></li><li><span><a href="#Load-drug-indication" data-toc-modified-id="Load-drug-indication-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Load drug-indication</a></span></li></ul></li><li><span><a href="#Convert-dict-to-df" data-toc-modified-id="Convert-dict-to-df-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Convert dict to df</a></span></li></ul></div>

# Imports

In [1]:
import time
import requests
import re
import os
import zipfile
import xml.etree.ElementTree
import pickle
import pandas as pd
import gzip

from tqdm import tqdm_notebook as tqdm

# Util Functions

## Save to pickle

In [2]:
def save_to_pickle(file_path, data):
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

## Load Pickle

In [3]:
def load_pickle(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        return data

## Open Zipfile

In [4]:
def open_zipfile(zip_path, file_name):
    archive = zipfile.ZipFile(zip_path, 'r')
    db_file = archive.open(file_name) 

# Load drug_list

In [8]:
drug_list_path = '../Data sets/DrugBank/drugbank_all_full_database.xml.zip'
inside_file_name = 'full database.xml'
drug_set_approved = set()
drug_set_not_approved = set()
archive = zipfile.ZipFile(drug_list_path, 'r')
db_file = archive.open(inside_file_name)

## Load Root

In [9]:
print('Starting to parse XML file')
root = xml.etree.ElementTree.parse(db_file).getroot()

Starting to parse XML file


In [10]:
ns = '{http://www.drugbank.ca}'
for i, drug in tqdm(enumerate(root)):
    assert drug.tag == '{ns}drug'.format(ns=ns)
    drug_p_id = drug.findtext("{ns}drugbank-id[@primary='true']".format(ns=ns))
    assert drug_p_id not in drug_set_approved and drug_p_id not in drug_set_not_approved
    assert len(drug.findall("{ns}groups".format(ns=ns))) == 1
    drug_approved=False
    for i,group in enumerate(drug.findall("{ns}groups".format(ns=ns))[0].getchildren()):
        if 'approved' == group.text:
            drug_approved = True
            break
    if drug_approved:
        drug_set_approved.add(drug_p_id)
    else:
        drug_set_not_approved.add(drug_p_id)


/home/rdo/.conda/envs/new_env/lib/python3.6/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [11]:
print(f'Approved drugs: {len(drug_set_approved)}')
print(f'Not Approved drugs: {len(drug_set_not_approved)}')

Approved drugs: 3988
Not Approved drugs: 9487


In [12]:
all_drugs = drug_set_approved.union(drug_set_not_approved)
drug_list = list(all_drugs)

## Save drugs list

In [13]:
saved_drugs_path = 'all_drugs.pickle'

In [14]:
save_to_pickle(saved_drugs_path, drug_list)

NameError: name 'savesaved_drugs_pathd_file_path' is not defined

## Load preprocessed file

In [14]:
drug_list = load_pickle(saved_drugs_path):

number of drugs: 3988


# Scrap

## Scrap drugs-indications

In [15]:
base_path = "https://www.drugbank.ca/drugs/"
# drug_list_path = 
# drug_list = ['DB00316','DB00787']
time_delay = 1

drug_ind_dict = {}
drug_info = "drug_info"
for d in tqdm(drug_list):
    url = base_path + d
    page = requests.get(url)
    inds = []
    for m_start in [m.start() for m in re.finditer('<li><a href="/indications/', page.text)]:
        m_end = page.text.find('">',m_start)
        m_text = page.text[m_start:m_end]
        indication_id = m_text[m_text.find('="/')+3 + len('indications/'):]
#         print(indication_id)
        
        m_start = page.text.find('">',m_end)
        m_end = page.text.find('</',m_start)
        m_text = page.text[m_start:m_end]
        indication_label = m_text[m_text.find('="/')+3 :]
#         print(indication_label)
        inds.append((indication_id, indication_label))
    drug_ind_dict[d] = inds
    time.sleep(time_delay)  # Delay for time_delay seconds.
#     file = open(os.path.join(drug_info, str(d)+".txt"), "w")
#     file.write(page.text)
#     file.close()
print(drug_ind_dict)

#download all pages of indictions with textual explanations

/home/rdo/.conda/envs/new_env/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Scap all drugs pages

In [69]:
base_path = "https://www.drugbank.ca/drugs/"
drugs_dir = "/home/rdo/Thesis/Data sets/DrugBank/All pages/"
time_delay = 1

# ans = {}
drug_info = "drug_info"
for d in tqdm(drug_list):
    url = base_path + d
    page = requests.get(url)
    file_path = os.path.join(drugs_dir, f'{d}.txt')
    with open(file_path, 'w') as file:
        file.write(page.text)
    

/home/rdo/.conda/envs/new_env/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


## Scrap all indication pages

In [70]:
base_path = "https://www.drugbank.ca/indications/"
drugs_dir = "/home/rdo/Thesis/Data sets/DrugBank/All indications/"

time_delay = 1

indication_list = df['ind_id'].unique()

# ans = {}
drug_info = "drug_info"
for d in tqdm(indication_list):
    url = base_path + d
    page = requests.get(url)
    file_path = os.path.join(drugs_dir, f'{d}.txt')
    with open(file_path, 'w') as file:
        file.write(page.text)

/home/rdo/.conda/envs/new_env/lib/python3.6/site-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


## save drugs and indications

In [16]:
drug_ind_path = 'drugs_indications_dict.pickle'
save_to_pickle(drug_ind_path, drug_ind_dict)

## Load drug-indication

In [6]:
drug_ind_dict = load_pickle(drug_ind_path)
len(drug_ind_dict)

13475

# Convert dict to df

In [32]:
data = [(key, ind[0],ind[1]) for key, indications in drug_ind_dict.items() for ind in indications]
data

[('DB00412', 'DBCOND0029752', 'Type 2 Diabetes Mellitus'),
 ('DB00485', 'DBCOND0015480', 'Animal bite'),
 ('DB00485', 'DBCOND0000235', 'Impetigo'),
 ('DB00485', 'DBCOND0021394', 'Infection caused by staphylococci'),
 ('DB00294', 'DBCOND0020891', 'Contraception'),
 ('DB00294', 'DBCOND0024304', 'Contraceptive implant therapy'),
 ('DB00006', 'DBCOND0034735', 'Acute Coronary Syndromes (ACS)'),
 ('DB00006', 'DBCOND0094088', 'Thrombotic events'),
 ('DB09526', 'DBCOND0023820', 'Melasma'),
 ('DB09526', 'DBCOND0025747', 'Moderate Melasma'),
 ('DB09526', 'DBCOND0025746', 'Severe Melasma'),
 ('DB04335', 'DBCOND0093803', 'Rejuvenation of a unit of RBC'),
 ('DB09280', 'DBCOND0040212', 'Cystic Fibrosis (CF)'),
 ('DB01193', 'DBCOND0075514', 'Chronic Stable Angina Pectoris'),
 ('DB01193', 'DBCOND0040789', 'High Blood Pressure (Hypertension)'),
 ('DB01193', 'DBCOND0102378', 'Arrhythmia of ventricular origin'),
 ('DB01337', 'DBCOND0024487', 'Adjunct to general anesthesia therapy'),
 ('DB01337',
  'DBCON

In [44]:
df = pd.DataFrame(data, columns=['drug', 'ind_id', 'ind_lbl'])
df['approved'] = df['drug'].apply(lambda x: 1 if x in drug_set_approved else 0)
print(df.shape)
df.head(10)

(8176, 4)


,drug,ind_id,ind_lbl,approved
0,DB00412,DBCOND0029752,Type 2 Diabetes Mellitus,1
1,DB00485,DBCOND0015480,Animal bite,1
2,DB00485,DBCOND0000235,Impetigo,1
3,DB00485,DBCOND0021394,Infection caused by staphylococci,1
4,DB00294,DBCOND0020891,Contraception,1
5,DB00294,DBCOND0024304,Contraceptive implant therapy,1
6,DB00006,DBCOND0034735,Acute Coronary Syndromes (ACS),1
7,DB00006,DBCOND0094088,Thrombotic events,1
8,DB09526,DBCOND0023820,Melasma,1
9,DB09526,DBCOND0025747,Moderate Melasma,1


In [48]:
df.to_csv('../Data sets/DrugBank/drugs_indications_df.csv')

In [41]:
print(f'unique drugs {df["drug"].nunique()}')
print(f'unique indications {df["ind_id"].nunique()}')

unique drugs 2052
unique indications 3539


In [43]:
print(f'Approved drugs: {len(drug_set_approved)}')
print(f'Not Approved drugs: {len(drug_set_not_approved)}')

Approved drugs: 3988
Not Approved drugs: 9487


In [46]:
df[df['approved'] == 1]['drug'].nunique()

2023